[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
https://colab.research.google.com/github/tailvar/theAIE/blob/master/aiecode/ch04/Capstone/notebook/incident_mcp_colab_demo.ipynb)

# Incident MCP Capstone — Colab Demo (Optional)

This notebook runs the **Incident Command Agent** end-to-end inside Colab.

- **Canonical workflow remains local** (recommended).
- This notebook is an **optional** demo path.

It runs the agent, which **spawns the stdio MCP server as a subprocess** and communicates via newline-delimited JSON-RPC over pipes.

This notebook does not change how the Incident Command Agent works. Colab is used purely as a lightweight execution environment. The 
agent and MCP server communicate using the same stdio-based JSON-RPC protocol as in local execution.

**For simplicity and reproducibility, the deterministic rules planner is used by default. LLM-based planning is supported but requires
external API keys and is therefore optional in Colab.**


## Why this notebook does “bootstrapping” only in Colab

This notebook is meant to be a *thin demo* for Capstone 4 — it should run the project the same way it would run locally i.e. (`python -m incident_mcp ...`) without changing the real repository state.

### Local Jupyter (running on "local" machine)
When notebook runs locally, repository is already saved to disk. Therefore the expected directories already exist:

- `config/` and `data/` are part of the repo
- `artifacts/` is usually already present (or created by normal runs)
- your file layout reflects the “real” project structure

So **the notebook should not create or modify directories locally**, because doing so can hide genuine setup issues or create files not intended to be added to git.

### Google Colab (ephemeral runtime)
Colab starts from a clean Linux machine. Nothing exists until its created:

- the repo must be cloned into the session
- output directories like `artifacts/` won’t exist unless they are created
- the memory log (`data/memory/memory.jsonl`) won’t exist until its created

So in Colab theres a small amount of **bootstrapping** to make sure the demo can run end-to-end.

### Key idea
We only “bootstrap” in Colab to compensate for its ephemeral filesystem, while keeping local execution canonical and reproducible.


## Where is notebook being run?

In [2]:
import os
from pathlib import Path

# Detect Colab
try:
    import google.colab  # type: ignore
    IN_COLAB = True
except Exception:
    IN_COLAB = False

cwd = Path.cwd().resolve()

def find_project_root(start: Path) -> Path:
    for parent in [start] + list(start.parents):
        if (parent / "pyproject.toml").exists() and (parent / "src").exists():
            return parent
    raise RuntimeError(f"Could not locate project root above {start}")

# Prefer robust detection; fallback to notebook heuristic
try:
    ROOT = find_project_root(cwd)
except RuntimeError:
    ROOT = cwd.parent if cwd.name == "notebook" else cwd

os.environ["INCIDENT_MCP_ROOT"] = str(ROOT)

print("IN_COLAB =", IN_COLAB)
print("Notebook cwd =", cwd)
print("INCIDENT_MCP_ROOT =", ROOT)



IN_COLAB = False
Notebook cwd = /home/oem/PycharmProjects/theAIE/aiecode/ch04/Capstone/notebook
INCIDENT_MCP_ROOT = /home/oem/PycharmProjects/theAIE/aiecode/ch04/Capstone


## 1) Install + fetch the project

In [3]:
# This notebook is an OPTIONAL demo harness for Capstone 4.
# It runs the Incident Command Agent exactly as it would run locally,
# using stdio-based JSON-RPC between agent and server.
"""
OPTIONAL SETUP CELL

• If running locally:
    - Do NOT clone the repo
    - Assume this notebook already lives inside the project tree

• If running in Google Colab:
    - Colab is a clean environment
    - Optionally clone the repo after explicit confirmation
"""

import os
from getpass import getpass

if IN_COLAB:
    print("Google Colab environment detected.")
    answer = input(
        "You are running in Colab.\n"
        "This will clone the repository into the Colab runtime.\n"
        "Type 'yes' to proceed, anything else to skip: "
    ).strip().lower()

    if answer == "yes":
        # Minimal dependencies required for Capstone 4
        !pip install -q python-dotenv

        # Optional: only needed for LLM planning
        !pip install -q anthropic openai

        # Clone the repository (Colab only)
        !git clone https://github.com/tailvar/theAIE.git
   
        # Move to Capstone directory
        %cd theAIE/aiecode/ch04/Capstone

        # Install the project in *editable* mode.
        #
        # This tells Python:
        #   "When importing `incident_mcp`, use the source code in ./src/"
        #
        # Editable installs work very well in notebooks and development:
        # - code changes picked up immediately
        # - no files are copied
        # - matches how project runs locally
        !pip install -e .
        
        # Set protocol root for MCP
        os.environ["INCIDENT_MCP_ROOT"] = os.getcwd()
        print("INCIDENT_MCP_ROOT =", os.environ["INCIDENT_MCP_ROOT"])
    else:
        raise RuntimeError("Repository clone skipped. Cannot proceed in Colab.")

else:
    print("Local Jupyter environment detected.")

Local Jupyter environment detected.


In [4]:
# --- Filesystem setup for Incident MCP Capstone ---
# Local Jupyter: assume directories already exist (DO NOTHING)
# Google Colab : create "expected" directories (ephemeral filesystem)

if IN_COLAB:
    print("Colab detected: bootstrapping filesystem structure.")

    # Create expected directories (rules + LLM use the same layout)
    (ROOT / "artifacts" / "traces").mkdir(parents=True, exist_ok=True)
    (ROOT / "artifacts" / "handoffs").mkdir(parents=True, exist_ok=True)
    (ROOT / "data" / "memory").mkdir(parents=True, exist_ok=True)

    # Ensure memory file exists (JSONL append-only)
    mem_path = ROOT / "data" / "memory" / "memory.jsonl"
    mem_path.touch(exist_ok=True)

    print("Artifacts dir:", ROOT / "artifacts")
    print("Memory file  :", mem_path)

else:
    print("Local environment detected.")
    print("Assuming filesystem already exists; no directories created.")


Local environment detected.
Assuming filesystem already exists; no directories created.


## 2) Choose planning mode

### Option A — Rules mode (recommended for Colab)
No API keys required, fully deterministic.

In [5]:
# --- Rules mode (default, simplest) ---
import os
os.environ["PLANNER_BACKEND"] = "rules"
print("PLANNER_BACKEND=", os.environ["PLANNER_BACKEND"])

PLANNER_BACKEND= rules


### Option B — Enable LLM-based planning (Anthropic or OpenAI)
Requires an API key. Use `getpass()` so you don't print secrets into outputs.
By default, this notebook uses the deterministic RULES planner so it can run without external API keys.

If you want to demonstrate LLM-based planning in Colab, uncomment ONE of the blocks below and provide your API key
securely using getpass().

NOTE:
 - Keys entered via getpass() are NOT stored in the notebook
 - They exist only for the lifetime of the Colab runtime
 - This mirrors best practice for ephemeral environments

In [6]:
if IN_COLAB:
    print(
        "Colab detected.\n"
        "If you want to run LLM mode, uncomment the optional cell below "
        "to provide API keys using getpass().\n"
        "If running locally, skip this entirely."
    )

### Optional: LLM API Keys (Colab only)
m
If you are running this notebook locally, **do not run the cell below**.
Local execution uses `.env` or shell environment variables.

If you are running in Google Colab and want to demonstrate LLM-based
planning, you may uncomment the cell below to enter your API key
securely using `getpass()`. Keys entered this way are not stored and
exist only for the lifetime of the Colab runtime.

In [7]:
# --- OPTIONAL: LLM mode (uncomment one block) ---

# (1) Anthropic
# from getpass import getpass
# os.environ["PLANNER_BACKEND"] = "anthropic"
# os.environ["ANTHROPIC_API_KEY"] = getpass("Enter ANTHROPIC_API_KEY: ")
# # Optional model override
# # os.environ["ANTHROPIC_MODEL"] = "claude-3-5-sonnet-20241022"

# (2) OpenAI
# from getpass import getpass
# os.environ["PLANNER_BACKEND"] = "openai"
# os.environ["OPENAI_API_KEY"] = getpass("Enter OPENAI_API_KEY: ")
# # Optional model override
# # os.environ["OPENAI_MODEL"] = "gpt-4o-mini"

## 3) Run the agent (agent spawns the server via stdio)

This is the key point: **you do not start the server separately** in stdio mode.
The agent starts it as a subprocess and talks JSON-RPC over stdin/stdout pipes.

In [8]:
!python -m incident_mcp agent --root "$INCIDENT_MCP_ROOT"

## 4) Inspect artifacts (telemetry, traces, memory, handoff)

In [9]:
cwd = Path.cwd().resolve()

# If you're inside Capstone/notebook, step up one level
ROOT = cwd.parent if cwd.name == "notebook" else cwd

# If you're not sure, you can harden this by looking for pyproject.toml
if not (ROOT / "pyproject.toml").exists() and (cwd / "pyproject.toml").exists():
    ROOT = cwd

# Export so your code + subprocesses agree
os.environ["INCIDENT_MCP_ROOT"] = str(ROOT)

# In notebooks, *also* change directory for shell commands like !ls
os.chdir(ROOT)

print("Notebook cwd now:", Path.cwd())
print("INCIDENT_MCP_ROOT:", os.environ["INCIDENT_MCP_ROOT"])

!echo "--- artifacts/ ---"; ls -R artifacts | head -n 200
print("--------------------------------------------------------------")
!echo "\n--- runs.jsonl (tail) ---"; tail -n 1 artifacts/runs.jsonl
print("--------------------------------------------------------------")
!echo "\n--- memory.jsonl (tail) ---"; tail -n 1 data/memory/memory.jsonl
print("--------------------------------------------------------------")
!echo "\n--- handoffs/ ---"; ls -l artifacts/handoffs || true
print("--------------------------------------------------------------")
!echo "\n--- latest handoff (if present) ---"; (ls -t artifacts/handoffs/*.md 2>/dev/null | head -n 1 | xargs -r sed -n '1,200p')

Notebook cwd now: /home/oem/PycharmProjects/theAIE/aiecode/ch04/Capstone
INCIDENT_MCP_ROOT: /home/oem/PycharmProjects/theAIE/aiecode/ch04/Capstone
--- artifacts/ ---
artifacts:
handoffs
runs.jsonl
traces

artifacts/handoffs:
handoff_run-1d94794224.md
handoff_run-48418aee93.md
handoff_run-87c4418aa9.md
handoff_run-9bf1ab6167.md

artifacts/traces:
run-029944aec7.jsonl
run-15662265c8.jsonl
run-179da361a4.jsonl
run-1d94794224.jsonl
run-24c28c0c5b.jsonl
run-2af2d3e411.jsonl
run-39e696da46.jsonl
run-48418aee93.jsonl
run-49c47816e1.jsonl
run-87c4418aa9.jsonl
run-8cc517464c.jsonl
run-901b29ec07.jsonl
run-996946ae60.jsonl
run-9bf1ab6167.jsonl
run-9ec55a73d7.jsonl
run-c96c8d95d4.jsonl
run-d3c19bc03d.jsonl
run-e44e19b43c.jsonl
run-eaa3c0860a.jsonl
--------------------------------------------------------------
\n--- runs.jsonl (tail) ---
{"run_id": "run-1d94794224", "step": 17, "event": "finish", "ts_utc": "2026-01-05T07:10:54Z", "budgets": {"steps_used": 17, "tool_calls_used": 3, "elapsed_ms": 63

## Notes / limitations

- Colab runtimes are **ephemeral**: artifacts disappear when the runtime resets.
- For persistence, mount Google Drive and point `artifacts/` and `data/` there.
- For a *networked* demo (local agent connecting to Colab server), use the **WebSocket server** and a tunnel (advanced).